#### Datansatz und Datenimport

In [190]:
import pandas as pd
import plotly.express as px

In [144]:
df = pd.read_parquet("DSCB310 - UE2 - Shopping Carts.parquet")

In [145]:
df.reset_index(inplace= True)

In [146]:
df.head()

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo


#### **P1:** Prüfen Sie, ob ein Zusammenhang zwischen dem Trinkgeldgeben und dem Vorhandensein eines der folgenden Produkte in einer Bestellung existiert: 24852, 24964, 2120

In [15]:
df.loc[df.product_id.isin([24852, 24964, 2120])]

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county
148,1726146,18,24964,29,1,56463,41,3,8,14.0,1,Organic Garlic,83,4,produce,fresh vegetables,Calaveras
239,3249426,22,24852,6,0,73241,1,0,15,NaN,0,Banana,24,4,produce,fresh fruits,Fresno
273,2769108,27,24852,3,1,130982,7,0,16,7.0,1,Banana,24,4,produce,fresh fruits,Plumas
333,1393512,34,24964,9,1,53965,25,5,9,4.0,1,Organic Garlic,83,4,produce,fresh vegetables,Calaveras
346,1393510,34,24852,1,1,53965,25,5,9,4.0,1,Banana,24,4,produce,fresh fruits,Calaveras
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6132129,5411136,606861,24852,2,1,177763,3,1,15,4.0,0,Banana,24,4,produce,fresh fruits,Mariposa
6132193,1987718,606868,24852,10,1,187352,41,3,11,3.0,1,Banana,24,4,produce,fresh fruits,Glenn
6132294,3738097,606880,24852,1,1,20236,2,5,13,30.0,0,Banana,24,4,produce,fresh fruits,Yolo
6132434,4417394,606892,24852,14,1,113114,41,0,21,8.0,1,Banana,24,4,produce,fresh fruits,Merced


In [69]:
pd.crosstab(df.product_id.isin([24852, 24964, 2120]), df.tip, margins= True)

tip,0,1,All
product_id,,,
False,3335155,2686369,6021524
True,53167,57978,111145
All,3388322,2744347,6132669


In [26]:
pd.crosstab(df.product_id.isin([24852, 24964, 2120]), df.tip, margins= True, normalize= "index")

tip,0,1
product_id,,
False,0.553872,0.446128
True,0.478357,0.521643
All,0.552504,0.447496


In [59]:
df_tip1 = df.loc[df.product_id.isin([24852, 24964, 2120])][["product_id", "tip"]].reset_index(drop= True)
df_tip1.head()

,product_id,tip
0,24964,1
1,24852,0
2,24852,1
3,24964,1
4,24852,1


In [65]:
df_tip1.loc[df_tip1.tip == 1]["tip"].count() / df_tip1.tip.count()

0.5216428989158307

In [66]:
df_tip1.loc[df_tip1.tip == 0]["tip"].count() / df_tip1.tip.count()

0.47835710108416934

In [60]:
df_tip2 = df.loc[~df.product_id.isin([24852, 24964, 2120])][["product_id", "tip"]].reset_index(drop= True)
df_tip2.head()

,product_id,tip
0,13176,1
1,11109,1
2,10246,1
3,22035,1
4,49683,1


In [64]:
df_tip2.loc[df_tip2.tip == 1]["tip"].count() / df_tip2.tip.count()

0.446127757690578

In [63]:
df_tip2.loc[df_tip2.tip == 0]["tip"].count() / df_tip2.tip.count()

0.553872242309422

In [67]:
df_tip1.corr()

,product_id,tip
product_id,1.000000,-0.076068
tip,-0.076068,1.000000


In [68]:
df_tip2.corr()

,product_id,tip
product_id,1.000000,-0.000941
tip,-0.000941,1.000000


In [70]:
# Chi Quadrat Test

#### **P2:** Lassen sich regionale Unterschiede im Trinkgeldverhalten erkennen?

In [77]:
df_county = df.groupby(["county"]).agg(tip_True =("tip", "sum"), tip_count =("tip", "count")).head()

In [170]:
pd.crosstab(df.county, df.tip, margins= True, normalize= "index")

tip,0,1
county,,
Alameda,0.581993,0.418007
Alpine,0.551991,0.448009
Amador,0.543910,0.456090
Calaveras,0.543822,0.456178
Colusa,0.514352,0.485648
Contra Costa,0.553613,0.446387
Del Norte,0.478250,0.521750
El Dorado,0.469429,0.530571
Fresno,0.533867,0.466133


In [239]:
df_tip_county = pd.crosstab(df.county, df.tip, margins= True, normalize= "index").reset_index()

In [256]:
df_tip_county.head()

tip,county,0,1
0,Alameda,0.581993,0.418007
1,Alpine,0.551991,0.448009
2,Amador,0.543910,0.456090
3,Calaveras,0.543822,0.456178
4,Colusa,0.514352,0.485648


In [189]:
df_tip_county[df_tip_county[0] < df_tip_county[1]]

tip,county,0,1
6,Del Norte,0.478250,0.521750
7,El Dorado,0.469429,0.530571
10,Humboldt,0.481175,0.518825
16,Lassen,0.496638,0.503362
23,Modoc,0.467365,0.532635
24,Mono,0.495229,0.504771
34,San Bernardino,0.493243,0.506757
35,San Diego,0.439309,0.560691
40,Santa Barbara,0.461810,0.538190
53,Tuolumne,0.474074,0.525926


In [255]:
df_tip_county_sorted = df_tip_county.sort_values(by= [1], ascending= False)
px.bar(x= df_tip_county_sorted.county, y= df_tip_county_sorted[1])

#### **P3:** Welche Attribute einer Bestellung wirken sich auf das Trinkgeldverhalten aus?

#### **P4:** Spielt die Vergangenheit eines Users eine Rolle in Hinblick auf die Trinkgeldwahrscheinlichkeit, oder kommt es nur auf Inhalt und Parameter der aktuellen Bestellung an?